In [1]:
%load_ext dotenv
%dotenv


In [2]:
import pandas as pd
import os
import googlemaps
from tqdm import tqdm
tqdm.pandas(desc="Geocoding..")

In [3]:
apikey = os.getenv("GMAP_API_KEY")
assert apikey is not None, "Please set GMAP_API_KEY in .env file"

In [4]:
gmaps = googlemaps.Client(key=apikey)

In [5]:
# create file cache
import joblib

cache=joblib.Memory(".cache", verbose=0)


@cache.cache
def geocode(street_address, subdistrict, district=None, province=None, country="TH"):
    # components = {'administrative_area_level_1': province, 'sublocality_level_1': district, 'sublocality_level_2': subdistrict, 'country': country}
    # # remove None values
    # components = {k: v for k, v in components.items() if v is not None}

    components = {'sublocality_level_2': subdistrict, 'country': country}
    if district is not None:
        components['sublocality_level_1'] = district
    if province is not None:
        components['administrative_area_level_1'] = province

    return gmaps.geocode(street_address, language="th", components=components)




In [17]:
geocode(street_address="โรงเรียนวัดธรรมมงคล (หลวงพ่อวิริยังค์อุปถัมภ์)", subdistrict="พระโขนง", district="บางจาก", province="กรุงเทพมหานคร")

[{'address_components': [{'long_name': '132',
    'short_name': '132',
    'types': ['street_number']},
   {'long_name': 'ซอย ปุณณวิถี',
    'short_name': 'ซอย ปุณณวิถี',
    'types': ['route']},
   {'long_name': 'แขวงบางจาก',
    'short_name': 'แขวงบางจาก',
    'types': ['political', 'sublocality', 'sublocality_level_2']},
   {'long_name': 'เขตพระโขนง',
    'short_name': 'เขตพระโขนง',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'กรุงเทพมหานคร',
    'short_name': 'กรุงเทพมหานคร',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'ประเทศไทย',
    'short_name': 'TH',
    'types': ['country', 'political']},
   {'long_name': '10260', 'short_name': '10260', 'types': ['postal_code']}],
  'formatted_address': '132 ซอย ปุณณวิถี แขวงบางจาก เขตพระโขนง กรุงเทพมหานคร 10260 ประเทศไทย',
  'geometry': {'location': {'lat': 13.6885237, 'lng': 100.6154805},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 13.6897808302

In [6]:
df = pd.read_csv('ballot_complete_clean.csv.gz')

In [7]:
df.columns

Index(['รหัสจังหวัด', 'จังหวัด', 'รหัสสำนักทะเบียน', 'สำนักทะเบียน',
       'รหัสตำบล', 'ตำบล', 'เขตเลือกตั้ง', 'หน่วยเลือกตั้ง',
       'สถานที่เลือกตั้ง', 'อธิบายเพิ่มเติมสถานที่เลือกตั้ง', 'NER'],
      dtype='object')

In [8]:
sample = df.sample(n=100)

In [9]:
sample['geocoded_obj'] = sample.progress_apply(lambda x: geocode(street_address=x['สถานที่เลือกตั้ง'], subdistrict=x["ตำบล"], province=x["จังหวัด"]), axis=1)

Geocoding..: 100%|██████████| 100/100 [00:11<00:00,  8.68it/s]


In [10]:
# check it is empty list
sample.geocoded_obj.apply(lambda x: len(x)).value_counts()

geocoded_obj
1    87
2    10
0     3
Name: count, dtype: int64

In [11]:
sample.to_excel("sample_geocoded.xlsx")

In [12]:
sample.to_pickle("sample_geocoded.pkl.gz")